# Détecteur des émotions par les expressions faciales

Un détecteur des émotions permet de détecter et d’analyser les émotions capturées à un instant t à partir d’une 
simple photo où une vidéo. Ce service peut identifier jusqu’à 7 émotions : la colère, le dégoût, la peur, le bonheur, neutre, la tristesse et la surprise.
Vous allez construire un modèle IA qui permet de réaliser cette tâche sur des images et des vidéos.



### Contexte du projet

Les expressions du visage peuvent naturellement servirent à évaluer la satisfaction d’un client aux prises avec un service après-vente ou à face à un produit récemment acquis dont il s’agit de comprendre le fonctionnement. On peut encore mentionner les applications suivantes :

    La détection d’un manque d’attention chez un conducteur en vue d’augmenter la sécurité de la conduite.
    L’évaluation du niveau de stress de passagers à l’atterrissage ou à l’arrivé en gare ou la détection de comportements suspects.
    L’humanisation des robots dans leurs interactions avec les humains dont ils prendraient en compte l’état psychique.

​




Les expressions faciales humaines peuvent être facilement classées en 7 émotions de base: heureuse, triste, surprise, peur, colère, dégoût et neutre.
    Nos émotions faciales sont exprimées par l'activation d'ensembles spécifiques de muscles faciaux. 
    Ces signaux parfois subtils, mais complexes, dans une expression contiennent souvent une quantité abondante 
    d'informations sur notre état d'esprit. Grâce à la reconnaissance des émotions faciales, nous sommes en mesure 
    de mesurer les effets du contenu et des services sur le public / les utilisateurs grâce à une procédure simple 
    et peu coûteuse. Par exemple, les détaillants peuvent utiliser ces mesures pour évaluer l'intérêt des clients. 
    Les prestataires de soins de santé peuvent fournir un meilleur service en utilisant des informations supplémentaires 
    sur l'état émotionnel des patients pendant le traitement. Les producteurs de divertissement peuvent surveiller 
    l'engagement du public dans les événements pour créer de manière cohérente le contenu souhaité.
    
    
Les émojis ou avatars sont des moyens d'indiquer des signaux non verbaux. Ces indices sont devenus une partie 
essentielle du chat en ligne, de l'examen des produits, de l'émotion de la marque et bien d'autres. 
Cela a également conduit à une augmentation de la recherche en science des données dédiée à la narration basée 
sur les emoji.

Grâce aux progrès de la vision par ordinateur et de l'apprentissage en profondeur, il est désormais possible de 
détecter les émotions humaines à partir d'images. Dans ce projet d'apprentissage en profondeur, nous classerons 
les expressions faciales humaines pour filtrer et mapper les emojis ou avatars correspondants.





L'ensemble de données reconnaissance des expressions faciales) se compose d'images de visage en niveaux de gris 
de 48 * 48 pixels. Les images sont centrées et occupent un espace égal. Cet ensemble de données comprend les 
émotions faciales des catégories suivantes:

    0: colère
    1: dégoût
    2: peur
    3: heureux
    4: neutre
    5: triste
    6: surpris
        
        



Nous allons créer un modèle d'apprentissage en profondeur pour classer les expressions faciales à partir des 
images. Ensuite, nous mapperons l'émotion classée à un emoji ou un avatar.



# Reconnaissance des émotions faciales à l'aide de CNN

Dans les étapes ci-dessous, on va construire une architecture de réseau de neurones à convolution et on va entraîner le
modèle sur l'ensemble de données pour la reconnaissance d'émotion à partir d'images.




In [6]:
#importation des bibliothèques

import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


In [5]:
#initialisation des générateurs train et test

train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Construction de l'architecture du réseau de convolution:


Le calque d'entrée a des dimensions prédéterminées et fixes, de sorte que l'image doit être prétraitée avant de 
pouvoir être introduite dans le calque. Le haar-cascade_frontalface_default.xml dans OpenCV contient des filtres 
pré-entraînés et utilise Adaboost pour trouver et rogner rapidement le visage.
Le visage rogné est ensuite converti en niveaux de gris à l'aide de cv2.cvtColor et redimensionné à 48 par 48 pixels
avec cv2.resize. Cette étape réduit considérablement les dimensions par rapport au format RVB d'origine avec 
trois dimensions de couleur (3, 48, 48). Le pipeline garantit que chaque image peut être introduite dans la 
couche d'entrée sous la forme d'un tableau numpy (1, 48, 48).

Le tableau numpy est passé dans la couche Convolution2D où je spécifie le nombre de filtres comme l'un des 
hyperparamètres. L'ensemble de filtres (aka. Kernel) est unique avec des poids générés aléatoirement. 
Chaque filtre, (3, 3) champ réceptif, glisse sur l'image d'origine avec des pondérations partagées pour créer 
une carte des caractéristiques.
La convolution génère des cartes d'entités qui représentent la manière dont les valeurs de pixel sont améliorées,
par exemple, la détection des contours et des motifs. Une carte des caractéristiques est créée en appliquant le 
filtre 1 sur toute l'image. D'autres filtres sont appliqués les uns après les autres pour créer un ensemble de 
cartes d'entités.

* Le pooling est une technique de réduction de dimension généralement appliquée après une ou plusieurs couches 
convolutives. Il s'agit d'une étape importante lors de la construction de CNN car l'ajout de couches convolutives
peut grandement affecter le temps de calcul. On utilise une méthode de mise en commun populaire appelée 
MaxPooling2D qui utilise (2, 2) fenêtres sur la carte des caractéristiques en ne gardant que la valeur maximale 
des pixels. Les pixels regroupés forment une image dont les dimensions sont réduites de 4.

* La couche dense (Dense Layer)  (aka couches entièrement connectées), est inspirée par la façon dont les 
neurones transmettent les signaux à travers le cerveau. Il prend un grand nombre d'entités d'entrée et 
transforme des entités à travers des couches connectées avec des poids entraînables.









In [ ]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

### Compilation et entraînement du modèle

In [10]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

C:\Users\utilisateur\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
448/448 [==============================] - 499s 1s/step - loss: 1.8308 - accuracy: 0.2391 - val_loss: 1.7282 - val_accuracy: 0.3298
Epoch 2/50
448/448 [==============================] - 301s 671ms/step - loss: 1.6779 - accuracy: 0.3387 - val_loss: 1.5496 - val_accuracy: 0.4102
Epoch 3/50
448/448 [==============================] - 300s 669ms/step - loss: 1.5648 - accuracy: 0.3953 - val_loss: 1.4948 - val_accuracy: 0.4305
Epoch 4/50
448/448 [==============================] - 298s 665ms/step - loss: 1.4888 - accuracy: 0.4231 - val_loss: 1.4175 - val_accuracy: 0.4668
Epoch 5/50
448/448 [==============================] - 310s 691ms/step - loss: 1.4159 - accuracy: 0.4579 - val_loss: 1.3809 - val_accuracy: 0.4731
Epoch 6/50
448/448 [==============================] - 300s 671ms/step - loss: 1.3682 - accuracy: 0.4798 - val_loss: 1.3315 - val_accuracy: 0.4948
Epoch 7/50
448/448 [==============================] - 303s 676ms/step - loss: 1.3344 - accuracy: 0.4920 - val_loss: 1.2913 - va

On obtient une accuracy de notre modèle de 87%, ce qui est un résultat plutôt bon.

### Sauvegarde du modèle pour être utilisé dans l'interface graphique

In [ ]:
emotion_model.save_weights('model.h5')

### En utilisant openCV haarcascade xml, on définit les boîtes englobantes du visage dans la webcam et on prédit les émotions:



In [ ]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Colère", 1: "Dégoût", 2: "Peur", 3: "Heureux", 4: "Neutre", 5: "Triste", 6: "Surpris"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2gray)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

    

### autre version

In [ ]:
# empêche l'utilisation d'openCL et les messages de journalisation inutiles
cv2.ocl.setUseOpenCL(False)

# dictionnaire qui attribue à chaque étiquette une émotion (ordre alphabétique)

emotion_dict = {0: "Colère", 1: "Dégoût", 2: "Peur", 3: "Heureux", 4: "Neutre", 5: "Triste", 6: "Surpris"}

# démarrer le flux webcam

cap = cv2.VideoCapture(0)
while True:
# Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Dans ce projet d'apprentissage en profondeur, nous avons construit un réseau neuronal à convolution pour 
reconnaître les émotions faciales.On a formé notre modèle et on a  cartographié ces émotions avec les emojis.

En utilisant le fichier XML en cascade de haar d'OpenCV, nous obtenons la boîte englobante des visages dans la 
webcam. Ensuite, nous transmettons ces boîtes au modèle entraîné pour la classification.



